In [58]:
import pandas as pd

qualifying_df = pd.read_csv("./raw_csv/qualifying.csv")
races_df = pd.read_csv("./raw_csv/races.csv")
constructors_df = pd.read_csv("./raw_csv/constructors.csv")
cstand_df = pd.read_csv("./raw_csv/constructor_standings.csv")


def add_features_years(x, start, end):
    df = qualifying_df.merge(races_df, "inner", on="raceId")
    df = df.merge(constructors_df, "inner", on="constructorId")
    df["q1"] = df["q1"].fillna(df["q2"])
    df["q1"] = df["q1"].fillna(df["q3"])
    df["q1"] = df["q1"].fillna("3:30.000")
    df = df[["year", "circuitId", "constructorId", "position", "q1", "name_y"]]
    for year in range (start, end+1):
        add = {"year": year}
        current_year = df[df["year"] == year]
        circuit_ids = current_year['circuitId'].tolist()
        for circuit_id in circuit_ids:
            current_circuit = current_year[current_year["circuitId"] == circuit_id]
            constructor_ids = current_circuit['constructorId'].tolist()
            for constructor_id in constructor_ids:
                current_constructor = current_circuit[current_circuit["constructorId"] == constructor_id]
                if current_constructor.size == 6:
                    add[f"circuit_{circuit_id}_team_{constructor_id}_1"] = current_constructor.iloc[0].q1
                    continue
                if current_constructor.size < 12:
                    print(f"Skip {current_constructor.size} {year} {circuit_id} {constructor_id}")
                    continue
                add[f"circuit_{circuit_id}_team_{constructor_id}_1"] = current_constructor.iloc[0].q1
                add[f"circuit_{circuit_id}_team_{constructor_id}_2"] = current_constructor.iloc[1].q1
        x = x.append(add, ignore_index=True)
        x.to_csv(f"{start}_{end}_checkpoint_during_adding_quali.csv")
    return x


def add_year_winner(x):
    df = cstand_df[cstand_df["position"] == 1]
    df = df.merge(races_df, "inner", on="raceId")
    df = df.sort_values('round', ascending=False)
    df.groupby("year").first()
    df = df[["constructorId", "year"]]

    # print(df)
    # print(df[df["year"] == 2019]["constructorId"].iloc[0])

    x["winner"] = None
    col_index = x.columns.get_loc('winner')
    for index, row in x.iterrows():
        print(row["year"])
        x.iat[index, col_index] = df[df["year"] == row["year"]]["constructorId"].iloc[0]
    return x

start_year = 1994
end_year = 2021
# x = pd.DataFrame()
# x = add_features_years(x, start_year, end_year)
# x.to_csv(f"{start_year}_{end_year}_checkpoint_before_adding_year_winner.csv")
# x = pd.read_csv("./1950_2021_checkpoint_before_adding_year_winner.csv")
# x = add_year_winner(x)
# x.to_csv(f"{start_year}_{end_year}_ready.csv")


1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [61]:
df = pd.read_csv("./classification_csvs/2005_2015_ready.csv")
print(df)
df = df.replace('\:','',regex=True).astype(float)
df.to_csv("./classification_csvs/2005_2015_ready.csv")

    Unnamed: 0  year circuit_20_team_1_1 circuit_20_team_1_2  \
0            0  2005            1:30.197            1:30.890   
1            1  2006            1:31.263            1:31.774   
2            2  2007            1:31.074            1:31.587   
3            3  2008                 NaN                 NaN   
4            4  2009            1:31.473            1:31.881   
5            5  2010                 NaN                 NaN   
6            6  2011            1:32.934            1:33.224   
7            7  2012                 NaN                 NaN   
8            8  2013            1:31.181            1:31.498   
9            9  2014                 NaN                 NaN   
10          10  2015                 NaN                 NaN   

   circuit_20_team_6_1 circuit_20_team_6_2 circuit_20_team_7_1  \
0             1:31.249            1:31.585            1:30.700   
1             1:31.235            1:31.921            1:31.809   
2             1:31.522           